You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: The Latest Trends in Artificial Intelligence: A Comprehensive Guide

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of key players in the AI field and their contributions

Key Points:
1. Latest Trends in AI:
   - Natural Language Processing (NLP) advancements
   - Computer vision technology developments
   - AI-powered healthcare solutions
   - Autonomous vehicles and robotics
   - AI ethics and regulation updates

2

I now can give a great answer

Final Answer:

# The Latest Trends in Artificial Intelligence: A Comprehensive Guide

**Introduction:**

Artificial Intelligence (AI) continues to revolutionize industries and everyday life with advancements in machine learning and data analytics. Companies like Google DeepMind, IBM Watson, and Amazon Web Services (AWS) are driving innovation in AI research and development, reshaping healthcare, finance, transportation, and more.

**Latest Trends in AI:**

In 2021, Natural Language Processing (NLP) technologies have seen significant advancements, enabling machines to understand and generate human language accurately. This has led to new opportunities in customer service automation, content generation, and sentiment analysis. Additionally, computer vision technology has made strides in applications such as facial recognition and autonomous drones. AI-powered healthcare solutions are also improving patient outcomes with personalized treatment recommendation

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Latest Trends in Artificial Intelligence: A Comprehensive Guide

**Introduction:**

Artificial Intelligence (AI) continues to revolutionize industries and everyday life with advancements in machine learning and data analytics. Companies like Google DeepMind, IBM Watson, and Amazon Web Services (AWS) are driving innovation in AI research and development, reshaping healthcare, finance, transportation, and more.

**Latest Trends in AI:**

In 2021, Natural Language Processing (NLP) technologies have seen significant advancements, enabling machines to understand and generate human language accurately. This has led to new opportunities in customer service automation, content generation, and sentiment analysis. Additionally, computer vision technology has made strides in applications such as facial recognition and autonomous drones. AI-powered healthcare solutions are also improving patient outcomes with personalized treatment recommendations and predictive analytics.

Autonomous vehicles and robotics are key areas of focus, with AI algorithms enhancing transportation systems' safety and efficiency. The ethical implications of AI technologies are gaining attention, leading to updates in regulations to ensure responsible AI deployment. Staying informed about the latest trends in AI is crucial for businesses and individuals looking to leverage AI effectively.

**Key Players in the AI Industry:**

Google DeepMind, IBM Watson, Amazon Web Services (AWS), and Microsoft Azure are key players in the AI industry, driving innovation and offering scalable AI solutions. OpenAI, founded by Elon Musk and Sam Altman, focuses on developing artificial general intelligence for the benefit of humanity.

**Noteworthy News in AI:**

Recent breakthroughs in AI research, such as GPT-3's language generation capabilities and reinforcement learning algorithms, are reshaping the AI landscape. AI applications in finance, marketing, and cybersecurity are enhancing decision-making processes and operational efficiency. AI startups are also making waves with innovative solutions in predictive analytics, natural language understanding, and computer vision.

**Conclusion:**

As AI technologies advance, staying updated on the latest trends and key players in the industry is essential. By understanding AI's impact, engaging in discussions on ethical considerations, and following developments in the field, we can harness AI's potential for positive change. Subscribe to our newsletter for more insights on AI trends and join our social media platforms to be part of the AI conversation. Let's embrace the future of AI together and explore its endless possibilities!

*Stay informed, stay engaged, and explore the endless possibilities of Artificial Intelligence!*

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [17]:
topic = "How does Computing Vision assess the quality of the creative work?"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on How does Computing Vision assess the quality of the creative work?.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: How does Computing Vision assess the quality of creative work?

Introduction:
- Definition of Computing Vision in the context of assessing creative work
- Brief overview of the importance of using Computing Vision in evaluating creative work

Key Points:
1. Latest Trends:
- Advancements in Computing Vision technology for assessing creative work
- Integration of machine learning algorithms for more accurate evaluations
- Use of deep learning techniques to analyz

I now can give a great answer

Final Answer:
# How does Computing Vision Assess the Quality of Creative Work?

## Introduction

In the realm of assessing creative work, Computing Vision plays a crucial role in providing objective evaluations. Computing Vision refers to the field of computer science that focuses on enabling computers to interpret and understand the visual world. When applied to creative work assessment, Computing Vision utilizes advanced technology to analyze and evaluate visual content, providing insights and feedback based on data-driven algorithms rather than subjective opinions. This objective approach is essential in ensuring fair and accurate evaluations of creative work.

## Trends in Computing Vision for Creative Work Assessment

The latest trends in Computing Vision technology have seen significant advancements in the field. Machine learning algorithms are being integrated to enhance the accuracy of evaluations, allowing for more precise analysis of creative co

In [18]:
Markdown(result)

# How does Computing Vision Assess the Quality of Creative Work?

## Introduction

In the realm of assessing creative work, Computing Vision plays a crucial role in providing objective evaluations. Computing Vision refers to the field of computer science that focuses on enabling computers to interpret and understand the visual world. When applied to creative work assessment, Computing Vision utilizes advanced technology to analyze and evaluate visual content, providing insights and feedback based on data-driven algorithms rather than subjective opinions. This objective approach is essential in ensuring fair and accurate evaluations of creative work.

## Trends in Computing Vision for Creative Work Assessment

The latest trends in Computing Vision technology have seen significant advancements in the field. Machine learning algorithms are being integrated to enhance the accuracy of evaluations, allowing for more precise analysis of creative content. Additionally, deep learning techniques are being utilized to further improve the capabilities of Computing Vision systems in analyzing visual elements. These advancements have revolutionized the way creative work is assessed, providing a more efficient and reliable method for evaluating quality.

## Key Players in Computing Vision

Several key players in the Computing Vision landscape are leading the way in the development of technology for creative work assessment. Google Cloud Vision API, Amazon Rekognition, IBM Watson Visual Recognition, and Microsoft Azure Computer Vision are among the prominent platforms that offer sophisticated tools for analyzing visual content. These platforms leverage advanced algorithms and machine learning capabilities to provide comprehensive evaluations of creative work, catering to the needs of designers, artists, photographers, and marketing professionals.

## Noteworthy News in Computing Vision

Recent developments in Computing Vision have showcased the effectiveness of using this technology for creative work assessment. Case studies of companies utilizing Computing Vision for evaluation purposes have demonstrated the benefits of objective analysis in the creative process. Research findings have also highlighted the accuracy and efficiency of Computing Vision in evaluating visual content. Furthermore, ongoing updates and improvements in Computing Vision technology continue to enhance the capabilities of these systems, making them indispensable tools for creative professionals.

## Conclusion

In conclusion, Computing Vision offers a groundbreaking approach to assessing the quality of creative work. By leveraging advanced technology, machine learning algorithms, and deep learning techniques, Computing Vision provides objective evaluations that are free from subjective bias. As key players continue to innovate and improve their platforms, creative professionals can benefit from more efficient and accurate assessments of their work. For those looking to explore the possibilities of Computing Vision tools, there is a wealth of resources and opportunities available for further exploration and utilization.

---
This blog post provides a comprehensive overview of how Computing Vision assesses the quality of creative work, highlighting the importance of objective evaluations in the creative industry. By incorporating SEO keywords and structuring the content according to the provided outline, this piece is tailored to engage and inform the target audience of creative professionals, marketing professionals, and technology enthusiasts. With a focus on the latest trends, key players, and noteworthy news in Computing Vision, this blog post aims to educate and inspire readers to embrace the potential of advanced technology in their creative endeavors.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).